### Import 

In [1]:
import pandas as pd
from utilities_DL import get_DataSet_and_invalid_dates,get_MultiModel_loss_args_emb_opts
from DL_class import MultiModelTrainer, Trainer
from config import get_args
from paths import folder_path,file_name,get_save_directory

### Config

In [11]:
model_name = 'STGCN' #'CNN' 
args = get_args(model_name)

# Necessary args :
args.calendar_class = 3
args.embedding_dim = 3
args.type_calendar = 'tuple'
args.time_embedding = True
args.TE_transfer = True
# Other args: 
args.H = 0
args.W = 0
args.D = 0
args.L =args.H+args.W+args.D
args.single_station = True

args.specific_lr = False
args.K_fold = 1
args.multi_embedding= True
# Save Directory:
main_dir = get_save_directory(args)

### Load Model:

In [12]:
dataset,invalid_dates = get_DataSet_and_invalid_dates(folder_path,file_name,args.W,args.D,args.H,args.step_ahead,single_station = args.single_station)
(Datasets,DataLoader_list,time_slots_labels,dic_class2rpz,dic_rpz2class,nb_words_embedding) =  dataset.split_K_fold(args,invalid_dates)

# Load associated K_folds Models: 
(loss_function,Model_list,Optimizer_list,args_embedding) = get_MultiModel_loss_args_emb_opts(args,nb_words_embedding,dic_class2rpz,n_vertex = len(Datasets[0].columns))

model,optimizer,dataloader,dataset = Model_list[0], Optimizer_list[0],DataLoader_list[0],Datasets[0]

coverage period: 2019-01-01 00:00:00 - 2020-01-01 00:00:00
Time-step per hour: 4.0
! H+D+W = 0, which mean the Tensor U will be set to a Null vector
Ko: 3, enable padding: True
Blocks: [[1], [64, 16, 64], [64, 16, 64], [128, 128], [2]]


## Load du model en ne récupérant que les poids du TimeEmbedding Module:

In [13]:
trainer = Trainer(dataset,model,dataloader,args,optimizer,loss_function,scheduler = None, ray = False,args_embedding  =args_embedding,dic_class2rpz = dic_class2rpz, save_dir = None, fold = 0)
trainer.plot_bokeh_and_save_results(pd.DataFrame(),epoch=0,station=0)

Loading BokehJS ...

,CalendarClass,Position,Specific_lr,Type_calendar,Multi_Emb,epoch,PICP,MPIW,valid_loss,train_loss
0,3,input,False,tuple,True,0,0.432772,52.378315,None,None


## Load du module en récupérant l'ensemble des poids suavegardé:
Ne fonctionne que si le model est exactement identique (car les noms de chaque couche et leur structure doivent correspondre)

In [ ]:
model.load_state_dict(saved_checkpoint['state_dict'])
trainer = Trainer(dataset,model,dataloader,args,optimizer,loss_function,scheduler = None, ray = False,args_embedding  =args_embedding,dic_class2rpz = dic_class2rpz, save_dir = None, fold = 0)
trainer.plot_bokeh_and_save_results(pd.DataFrame(),epoch=0,station=0)

## Faisons Maintenant un Transfer Learning sur un Model totalement différent (mais avec les poids du TimeEmbedding adapté: )

In [ ]:
model_name = 'STGCN' #'CNN' 
args = get_args(model_name)

# Necessary args :
args.calendar_class = 3
args.embedding_dim = 3
args.type_calendar = 'tuple'
args.time_embedding = True

# Other args: 
args.epochs = 100
args.H = 6
args.W = 1
args.D = 1
args.L =args.H+args.W+args.D
args.single_station = True

args.specific_lr = False
args.K_fold = 1

# Save Directory:
main_dir = get_save_directory(args)

# Load Model 
dataset,invalid_dates = get_DataSet_and_invalid_dates(folder_path,file_name,args.W,args.D,args.H,args.step_ahead,single_station = args.single_station)
(Datasets,DataLoader_list,time_slots_labels,dic_class2rpz,dic_rpz2class,nb_words_embedding) =  dataset.split_K_fold(args,invalid_dates)
(loss_function,Model_list,Optimizer_list,args_embedding) = get_MultiModel_loss_args_emb_opts(args,nb_words_embedding,dic_class2rpz)
model,optimizer,dataloader,dataset = Model_list[0], Optimizer_list[0],DataLoader_list[0],Datasets[0]

# TE Transfer Learning
model.load_state_dict(embedding_weights, strict=False)
trainer = Trainer(dataset,model,dataloader,args,optimizer,loss_function,scheduler = None, ray = False,args_embedding  =args_embedding,dic_class2rpz = dic_class2rpz, save_dir = None, fold = 0)

# Training : 
trainer.train_and_valid(mod = 10000,mod_plot = 10, alpha = args.alpha,station = 0)

## Comparons maintenant avec un même model, mais sans le TransferLearning
C'est un cas où, nous l'avons vu, le TimeEmbedding ne sert à rien. Il n'est pas entrainé correctement.
- Après 1 epoch, l'avantage du Transfer Learning semble limité (Valid Loss 0.164 contre 0.170)
- Après 10 epochs, Idem avantage très limité (Valid Loss 0.107 contre 0.109)
- Après Seulement 20 epochs, le Transfer Learning n'a même plus d'intérêt (Valid Loss 0.099 contre 0.096). Mais on Voit aussi que l'espace d'embedding est déjà intéressant. 
- Après 100 epochs, le Transfer Learning n'a pas plus d'intérêt (Valid Loss 0.099 contre 0.084)

In [23]:
main_dir

'save/STGCN/K_fold1/H6_D1_W1/graph_conv/sym_norm_lap/act_glu_Ks2/optadamw/train_valid_calib_0.60.20.5/E100_lr0.0001_B64/FC1_17_8_FC2_8_4/Emb_dim3/Specific_lr_False/CalendarClass3/position_input/'

In [ ]:
# Load Model 
dataset,invalid_dates = get_DataSet_and_invalid_dates(folder_path,file_name,args.W,args.D,args.H,args.step_ahead,single_station = args.single_station)
(Datasets,DataLoader_list,time_slots_labels,dic_class2rpz,dic_rpz2class,nb_words_embedding) =  dataset.split_K_fold(args,invalid_dates)
(loss_function,Model_list,Optimizer_list,args_embedding) = get_MultiModel_loss_args_emb_opts(args,nb_words_embedding,dic_class2rpz)
model,optimizer,dataloader,dataset = Model_list[0], Optimizer_list[0],DataLoader_list[0],Datasets[0]

# TE Transfer Learning
# model.load_state_dict(embedding_weights, strict=False)
trainer = Trainer(dataset,model,dataloader,args,optimizer,loss_function,scheduler = None, ray = False,args_embedding  =args_embedding,dic_class2rpz = dic_class2rpz, save_dir = None, fold = 0)

# Training : 
trainer.train_and_valid(mod = 10000,mod_plot = 10, alpha = args.alpha,station = 0)

## Maintenant, essayons de Load un Multi-Station Model, mais en y mettant un time-embedding commun à toute stations:

In [ ]:
args.single_station = False
# Load Model 
dataset,invalid_dates = get_DataSet_and_invalid_dates(folder_path,file_name,args.W,args.D,args.H,args.step_ahead,single_station = args.single_station)
(Datasets,DataLoader_list,time_slots_labels,dic_class2rpz,dic_rpz2class,nb_words_embedding) =  dataset.split_K_fold(args,invalid_dates)
(loss_function,Model_list,Optimizer_list,args_embedding) = get_MultiModel_loss_args_emb_opts(args,nb_words_embedding,dic_class2rpz)
model,optimizer,dataloader,dataset = Model_list[0], Optimizer_list[0],DataLoader_list[0],Datasets[0]

# TE Transfer Learning
model.load_state_dict(embedding_weights, strict=False)
trainer = Trainer(dataset,model,dataloader,args,optimizer,loss_function,scheduler = None, ray = False,args_embedding  =args_embedding,dic_class2rpz = dic_class2rpz, save_dir = None, fold = 0)

# Training : 
trainer.train_and_valid(mod = 10000,mod_plot = 10, alpha = args.alpha,station = 0)